<a href="https://colab.research.google.com/github/kassio077/Aprendizado-de-maquina/blob/master/Atividade_6_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> # Classificação de imagens com Deep Learning

O dataset Moda MNIST consiste de imagens 28x28 de um único canal de cor, com 60000 exemplos para o conjunto de treinamento e 10000 para teste. Cada instância esta rotulada com uma das seguintes classes:

* 0 Camiseta / top
* 1 calça
* 2 pulôver
* 3 vestido
* 4 brasão
* 5 sandálias
* 6 Camisa 
* 7 tênis
* 8 saco
* 9 bota tornozelo

Cada linha do dataset representa uma imagem com 785 colunas. As colunas representam os pixels com valor de 0 a 255, a primeira coluna está rotulada com uma das classes.

Link para o dataset: [moda](https://www.kaggle.com/zalando-research/fashionmnist?resource=download)

#  Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

from sklearn.metrics import accuracy_score
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop

from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import RandomizedSearchCV

> # **1.Carregando os dados**

In [2]:
# lendo csv 
treino = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Moda/fashion-mnist_train.csv')
treino.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,0,0,105,92,101,107,100,132,0,0,2,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,150,...,211,220,214,74,0,255,222,128,0,0,0,0,0,0,0,0,0,44,12,0,0,40,134,162,191,214,163,146,165,79,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,0,114,183,112,55,23,72,102,165,160,28,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,24,188,163,93,...,171,249,207,197,202,45,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,22,21,25,69,52,45,74,39,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,0,0,0,46,0,21,68,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,187,189,...,230,237,229,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,68,116,112,136,147,144,121,102,63,0,0,0,0,0,0,0,0,0,0


In [ ]:
treino.shape

(60000, 785)

In [3]:

# lendo csv e armazenando em um dataframe
teste = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Moda/fashion-mnist_test.csv')
teste.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,0,0,34,29,7,0,11,24,0,0,3,3,1,0,1,1,0,0,0,0,0,4,0,0,1,0,0,0,0,0,...,242,245,224,245,234,0,3,0,0,0,0,0,0,0,0,0,0,0,0,37,69,94,123,127,138,138,142,145,135,125,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,209,190,181,150,170,193,180,219,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,...,248,238,80,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,174,233,155,0,65,235,216,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,17,0,0,0,0,0,0,0,0,12,94,68,14,0,0,0,0,0,0,0,0,0,0,0,38,106,94,89,94,68,...,0,0,0,0,0,0,87,12,26,0,0,0,0,0,0,0,46,50,29,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,161,212,138,150,169,164,176,202,255,183,26,0,0,0,0,0,0,0,0,0,0,0,1,0,0,129,221,255,255,215,...,255,255,254,252,255,101,196,254,252,100,0,0,0,0,0,168,232,225,0,41,172,147,148,153,155,146,137,141,143,137,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,0,37,0,0,0,0,0,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21,204,235,...,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,69,0,2,0,0,0,0,0,0,0,0,0,0


In [ ]:
teste.shape

(10000, 785)

> # **2. Adequando os dados para o modelo**

* Removendo a coluna de rótulos das instâncias

In [4]:
X_treino = treino.drop(columns=['label'])
y_treino = treino['label']

In [5]:
X_teste = teste.drop(columns=['label'])
y_teste = teste['label']


* Transformando a label das instâncias em vetores categóricos para representar a saída do modelo de classificação.

In [6]:
y_treino = to_categorical(y_treino, num_classes = 10)
y_teste = to_categorical(y_teste, num_classes = 10)

In [7]:
y_treino

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

> **Re-escala dos dados**

In [8]:
X_treino = X_treino/255
X_treino.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,pixel40,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.019608,0.0,0.0,0.000000,0.411765,0.360784,0.396078,0.419608,0.392157,0.517647,0.000000,0.000000,0.007843,0.015686,0.0,0.003922,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.588235,0.890196,...,0.827451,0.862745,0.839216,0.290196,0.000000,1.000000,0.870588,0.501961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.172549,0.047059,0.0,0.0,0.156863,0.525490,0.635294,0.749020,0.839216,0.639216,0.572549,0.647059,0.309804,0.000000,0.0,0.0,0.117647,0.168627,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.003922,0.007843,0.0,0.0,0.000000,0.0,0.0,0.447059,0.717647,0.439216,0.215686,0.090196,0.282353,0.400000,0.647059,0.627451,0.109804,0.000000,0.0,0.000000,0.003922,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003922,0.0,0.0,0.094118,0.737255,0.639216,0.364706,0.533333,...,0.670588,0.976471,0.811765,0.772549,0.792157,0.176471,0.000000,0.011765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.003922,0.0,0.0,0.000000,0.086275,0.082353,0.098039,0.270588,0.203922,0.176471,0.290196,0.152941,0.011765,0.0,0.0,0.000000,0.000000,0.003922,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.180392,0.000000,0.082353,0.266667,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.098039,0.733333,0.741176,0.000000,...,0.901961,0.929412,0.898039,0.000000,0.000000,0.003922,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.266667,0.454902,0.439216,0.533333,0.576471,0.564706,0.474510,0.400000,0.247059,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0


In [9]:
X_teste = X_teste/255
X_teste.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,pixel40,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.035294,0.031373,0.000000,0.000000,0.133333,0.113725,0.027451,0.000000,0.043137,0.094118,0.000000,0.000000,0.011765,0.011765,0.003922,0.0,0.003922,0.003922,0.0,0.0,0.0,0.0,0.0,0.015686,0.000000,0.0,0.003922,0.000000,0.000000,0.000000,0.000000,0.000000,0.172549,...,0.949020,0.960784,0.878431,0.960784,0.917647,0.000000,0.011765,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.145098,0.270588,0.368627,0.482353,0.498039,0.541176,0.541176,0.556863,0.568627,0.529412,0.490196,0.403922,0.341176,0.219608,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.819608,0.745098,0.709804,0.588235,0.666667,0.756863,0.705882,0.858824,0.019608,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.094118,0.921569,...,0.972549,0.933333,0.313725,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.682353,0.913725,0.607843,0.000000,0.254902,0.921569,0.847059,0.133333,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.054902,0.207843,0.388235,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047059,0.368627,0.266667,0.054902,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.149020,0.415686,0.368627,0.349020,0.368627,0.266667,0.007843,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.341176,0.047059,0.101961,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.180392,0.196078,0.113725,0.000000,0.007843,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.247059,0.207843,0.121569,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.631373,0.831373,0.541176,0.588235,0.662745,0.643137,0.690196,0.792157,1.000000,0.717647,0.101961,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.003922,0.0,0.000000,0.505882,0.866667,1.000000,1.000000,0.843137,0.721569,...,1.000000,1.000000,0.996078,0.988235,1.000000,0.396078,0.768627,0.996078,0.988235,0.392157,0.0,0.0,0.0,0.0,0.0,0.658824,0.909804,0.882353,0.000000,0.160784,0.674510,0.576471,0.580392,0.600000,0.607843,0.572549,0.537255,0.552941,0.560784,0.537255,0.537255,0.494118,0.549020,0.0,0.521569,0.878431,0.870588,0.219608,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.145098,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.082353,0.800000,0.921569,0.921569,...,0.000000,0.007843,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003922,0.000000,0.007843,0.270588,0.000000,0.007843,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0


Como a imagem representa um vetor de uma dimensão foi preciso remontar cada linha da tabela em formato (28 x 28 x 1).

In [10]:
# Reorganizando dados para um volume 28 x 28 x 1 (apenas um canal de cores)
X_treino = X_treino.values.reshape(-1,28,28,1)
X_teste = X_teste.values.reshape(-1,28,28,1)

> # **3. Definindo o modelo com CNN**

A principio criamos um modelo sequencial com 3 super-camadas e definimos algumas configurações, cada uma tem uma função específica no processo de construção do modelo.

In [14]:
#definição do modelo
model = Sequential()

> Camada de extração de característica

*   Camada de convolução com 32 filtros de tamanho 5x5 para receber os dados de dimensão 28x28x1; 
*   Camada de covolução com 32 filtros de tamanho 4x4;
*   Redução de dimensão para 14x14x1.



In [12]:
# 2 camadas de convolução de 32 filtros + Max pooling [2,2]
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (4,4),padding = 'Same', activation ='softmax'))
model.add(MaxPool2D(pool_size=(2,2)))


In [13]:
# 2 camadas de convolução de 64 filtros + Max pooling [2,2]
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='softmax'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

> Aplicamos a saída das camadas anteriores de extração de características uma camada composta por: 
* Conversão do vetor de 1 dimenção;
* Camada completamente conectada de 256 neurônios;
* Desativação de 50% dos neurônios;
* Uma camada completamente conectada que determinará a saída da rede a partir de um  vetor categórico com as probabilidades de cada uma das classes.

In [ ]:
#  2 camadas completamente conectadas
model.add(Flatten())#convert the final feature maps into a one single 1D vector
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

* Otmizador com alguns valores padrões:

In [ ]:
# Definindo o algoritmo otmizador
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-07, decay=0.0, centered=False)

In [ ]:
# Compilando o modelo
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        16416     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
dropout (Dropout)            (None, 7, 7, 64)         

Com estas configurações o modelo apresentou uma acurácia de 77% para as imagens de treino.  

In [ ]:
# treinamento do modelo
model.fit(X_treino, y_treino, verbose = 1)

1875/1875 [==============================] - 354s 189ms/step - loss: 0.6183 - accuracy: 0.7719


> **Avaliação**

Após o treinamento do modelo avaliamos os resutados da predição das imagens de teste. A acurácia foi 10% melhor em relação a de treino.

In [ ]:
model.evaluate(X_teste,y_teste)


313/313 [==============================] - 14s 44ms/step - loss: 0.3360 - accuracy: 0.8772


[0.3359980285167694, 0.8772000074386597]

>  **Modelo 2**

Criamos um cenário para CNN com algumas configurações diferentes, para as camadas de extração de caracteristicas aplicamos filtros de dimenções 5x5 e 3x3, função de ativação relu, e um Dropout de 20%.

> Super camada com filtros de mesmo formato e função de ativação, agora aplicamos um Dropout de 20%

In [15]:
model = Sequential()

In [16]:
# 2 camadas de convolução de 32 filtros + Max pooling [2,2]
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.20))

In [17]:
# 2 camadas de convolução de 64 filtros + Max pooling [2,2]
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.20))

In [18]:
#  2 camadas completamente conectadas
model.add(Flatten())#convert the final feature maps into a one single 1D vector
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [19]:
# Definindo o algoritmo otmizador
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-07, decay=0.0, centered=False)

In [21]:
# Compilando o modelo
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [22]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 64)         

Com este cenário de configurações nas camadas de extração o modelo apresentou uma melhor acurácia no treinamento.

In [ ]:
model.fit(X_treino, y_treino, verbose = 1)

1875/1875 [==============================] - 419s 224ms/step - loss: 0.4629 - accuracy: 0.8365


In [ ]:
## teste
#classificacao = model.predict(X_teste)

> **Avaliação**



Para avaliação das imagens de teste o resultado também foi superior, se compararmos ao primeiro teste, 89% das imagens classificadas corretamente. 

In [ ]:
model.evaluate(X_teste,y_teste)


313/313 [==============================] - 16s 52ms/step - loss: 0.2935 - accuracy: 0.8917


[0.29347285628318787, 0.891700029373169]

> # **4. Configurando hiperparâmetros no treinamento**

Com as configurações do último modelo avaliado aumentamos o número de épocas de treinamento para 3 com 3000 steps cada.

Ápos o treinamento a acurácia do modelo apresentou uma leve melhora de 83% para 89% para as imagens de treino. 

In [24]:
model.fit(X_treino,y_treino,
                         steps_per_epoch = 3000,
                         epochs = 3
                         )

Epoch 1/3
3000/3000 [==============================] - 427s 142ms/step - loss: 0.4559 - accuracy: 0.8411
Epoch 2/3
3000/3000 [==============================] - 425s 142ms/step - loss: 0.3661 - accuracy: 0.8737
Epoch 3/3
3000/3000 [==============================] - 430s 143ms/step - loss: 0.3733 - accuracy: 0.8739


> **Avaliando o modelo**

Para as predições das imagens de teste não tivemos melhora na acurácia o resultado foi praticamente o mesmo.

In [25]:
model.evaluate(X_teste,y_teste)

313/313 [==============================] - 16s 53ms/step - loss: 0.2967 - accuracy: 0.8920


[0.29667946696281433, 0.8920000195503235]

> **Treinando rede mais profunda**

Tentamos melhorar o classificador atribuindo mais camadas de convolução a rede. Agora adicionamos 3 camadas de convolução nas camadas de extração de características com filtros de 32 e 64 tamanho 3x3.

In [27]:
model = Sequential()

In [28]:
# 3 camadas de convolução de 32 filtros + Max pooling [2,2]
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

In [29]:
# 3 camadas de convolução de 64 filtros + Max pooling [2,2]
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

In [30]:
#  2 camadas completamente conectadas
model.add(Flatten())#convert the final feature maps into a one single 1D vector
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [31]:
# Definindo o algoritmo otmizador
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-07, decay=0.0, centered=False)

In [32]:
# Compilando o modelo
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [34]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 14, 14, 64)       

Para este cenário configuramos o treinamento com 3 épocas com 1900 passos cada, no entanto, não obtivemos melhora na acurácia do treino. Tivemos uma perda de passos durante a terceira época.

In [35]:
model.fit(X_treino,y_treino,
                         steps_per_epoch = 1900,
                         epochs = 3
                         )

Epoch 1/3
1900/1900 [==============================] - 417s 219ms/step - loss: 0.4537 - accuracy: 0.8399
Epoch 2/3
1900/1900 [==============================] - 417s 219ms/step - loss: 0.3165 - accuracy: 0.8907
Epoch 3/3
1825/1900 [===========================>..] - 394s 216ms/step - loss: 0.3082 - accuracy: 0.8947


> **Avaliação**

No teste a melhora da acurácia na classificação das imagens de teste foi discreta. 

In [36]:
model.evaluate(X_teste,y_teste)

313/313 [==============================] - 16s 52ms/step - loss: 0.2612 - accuracy: 0.9054


[0.2611715793609619, 0.9053999781608582]